In [ ]:
!pip install accelerate datasets peft evaluate bitsandbytes optuna packaging ninja transformers requests --upgrade
!pip install transformers --upgrade
# !MAX_JOBS=2 pip install flash-attn --no-build-isolation torch-xla

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from google.colab import drive
import pathlib
os.environ['WANDB_DISABLED'] = 'true'
os.environ['WANDB_MODE'] = 'disabled'
drive.mount('/content/drive')
# https://cf020031308.github.io/wiki/optuna/

Mounted at /content/drive


In [ ]:
# %%
import os
os.chdir('/content/drive/MyDrive/models')
import argparse
import json,hashlib
from transformers import AutoConfig, AutoModelForQuestionAnswering, AutoModel, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import TrainingArguments, Trainer
from transformers import DefaultDataCollator, DataCollatorWithPadding
import transformers
from torch import nn
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training, PeftModel
from datasets import IterableDataset #, DataLoader
import datasets
import numpy as np
import get_data
import torch
# from torch.utils.data import DataLoader
import importlib
import pathlib
import re
from typing import List
try:
    import requests
except:
    pass

importlib.reload(get_data)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
def to_gpu_tensor(batch, device=device):
    return {key: torch.tensor(value).to(device) for key, value in batch.items()}

parser = argparse.ArgumentParser(
    description="Example script",
    formatter_class=argparse.ArgumentDefaultsHelpFormatter
)
# parser.add_argument('--batch_size', type=int, default=1, help="batch_size")
# parser.add_argument('--question_filename', type=str, default="questions_example.json", help="question_filename")
# parser.add_argument('--write_path', type=str, default="/content/drive/MyDrive/models/predictions", help="write_path")
# parser.add_argument('--df_start_id', type=int, default=0, help="df_start_id")
# parser.add_argument('--df_end_id', type=int, default=None, help="df_start_id")
# parser.add_argument('--model_dir_name', type=str, default="815aace0d964e9dad6ee4899b9ce83efea759b914d0a352c4e0cce1ad052d481", help="model_dir_name")
# parser.add_argument('--checkpoint_num', type=str, default="120", help="checkpoint_num")
# parser.add_argument('--ground_truths_filename', type=str, default="ALL.json", help="checkpoint_num")
# parser.add_argument('--scan_ans_windowsize', type=int, default=20, help="scan_ans_windowsize")
# parser.add_argument('--requrl', type=str, default=None, help="invoke request")

# args = parser.parse_args()
# parserargs_dict = vars(args)
parserargs_dict = {
    'batch_size':1,
    'question_filename':'questions_preliminary.json',
    'write_path':'/content/drive/MyDrive/models/predictions',
    'df_start_id':500,
    'model_dir_name':'815aace0d964e9dad6ee4899b9ce83efea759b914d0a352c4e0cce1ad052d481',
    'checkpoint_num':600,
    'df_end_id':None,
    'ground_truths_filename':"ALL120.json",
    'scan_ans_windowsize':20,
    'requrl':None,
}


print(f"parserargs_dict is {parserargs_dict}")
writepath = pathlib.Path(parserargs_dict['write_path'])
model_base_dir = get_data.current_folder
for target_create_path in [writepath, model_base_dir]:
    target_create_path.mkdir(parents=True, exist_ok=True)

model_base_dir = model_base_dir/parserargs_dict['model_dir_name']
checkpoint_dir = model_base_dir/f"checkpoint-{parserargs_dict['checkpoint_num']}"
argp_dict = (model_base_dir/"argp_dict.json").read_text()
argp_dict = json.loads(argp_dict)
# %%
argp_dict['bnb_4bit_compute_dtype'] = torch.bfloat16 if argp_dict['bnb_4bit_compute_dtype']=='bf16' else torch.float32
if argp_dict['load_in_8bit'] in [0,None,False] and argp_dict['load_in_4bit'] in [0,None,False]:
    is_quantized = False
    quantization_config = None
else:
    is_quantized = True
    quantization_config = BitsAndBytesConfig(
        load_in_8bit=argp_dict['load_in_8bit'],
        load_in_4bit=argp_dict['load_in_4bit'],
        llm_int8_has_fp16_weight=argp_dict['llm_int8_has_fp16_weight'],
        bnb_4bit_use_double_quant=argp_dict['bnb_4bit_use_double_quant'],
        bnb_4bit_compute_dtype=argp_dict['bnb_4bit_compute_dtype'],
        bnb_4bit_quant_type=argp_dict['bnb_4bit_quant_type'],
    )

if argp_dict['peft_r'] in [0,None,False] or argp_dict['peft_lora_alpha'] in [0,None,False]:
    is_peft = False
    peft_config = None
else:
    is_peft = True
    peft_config = LoraConfig(
        task_type=TaskType.QUESTION_ANS,
        inference_mode=False,
        r=argp_dict['peft_r'],
        lora_alpha=argp_dict['peft_lora_alpha'],
        lora_dropout=0.1
    )

model = AutoModelForQuestionAnswering.from_pretrained(
    checkpoint_dir,
    token=get_data.access_token,
    trust_remote_code=True,
    quantization_config=quantization_config,
    attn_implementation=argp_dict['attn_implementation'],
    device_map="auto",
)
if is_quantized:
    model = prepare_model_for_kbit_training(model)
if is_peft:
    model = PeftModel.from_pretrained(model, checkpoint_dir, is_trainable=False)
print("model load complete")
# %%
drop_cols_1 = ['start_positions', 'end_positions']
drop_cols_2 = ['qid', 'question', 'category', 'context']

# %%

prediction_batch_size = parserargs_dict['batch_size']
get_data_instance = get_data.GetdataClass(model_name=argp_dict['model_name'])

model = model.to(device)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


parserargs_dict is {'batch_size': 1, 'question_filename': 'questions_preliminary.json', 'write_path': '/content/drive/MyDrive/models/predictions', 'df_start_id': 500, 'model_dir_name': '815aace0d964e9dad6ee4899b9ce83efea759b914d0a352c4e0cce1ad052d481', 'checkpoint_num': 480, 'df_end_id': None, 'ground_truths_filename': 'ALL120.json', 'scan_ans_windowsize': 20, 'requrl': None}


Some weights of Qwen2ForQuestionAnswering were not initialized from the model checkpoint at Qwen/Qwen2.5-0.5B-Instruct and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model load complete


In [ ]:
competition_dataframe = get_data_instance.get_competition_dataset_faq(join_truths=False, questions_filename=parserargs_dict['question_filename'])
# parserargs_dict['df_start_id'] = 110
competition_dataframe = competition_dataframe.sort_values(by=['qid'], ascending=False)#.iloc[0:parserargs_dict['df_start_id']].sort_values(by=['qid'], ascending=False)
testset = datasets.Dataset.from_pandas(competition_dataframe.iloc[parserargs_dict['df_start_id']:parserargs_dict['df_end_id']])

iterds_validation = testset \
    .map(get_data_instance.markup_article_id, batched=False, with_indices=False)

Map:   0%|          | 0/790 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=get_data_instance.tokenizer)
# dataloader = DataLoader(
#     iterds_validation.remove_columns(column_names=drop_cols_2),
#     batch_size=prediction_batch_size,
#     collate_fn=data_collator
# )
prediction_batch_size = 1
iterds_validation_batched = iterds_validation.batch(prediction_batch_size)

match_doc_id_pattern = re.compile(r'(?:insurance|finance|faq)_\d+')
def slice_list_by_list_of_int(srclist:List, indices:List)->List:
    returnlist = []
    for id in indices:
        returnlist.append(srclist[id])
    return returnlist
def model_output_arrange(model_outputs, batchdata=None, scan_ans_windowsize_limit:int=25)->List:
    global match_doc_id_pattern
    func_batch_predictions_for_competition = []
    max_seq_length = model_outputs.start_logits.shape[1]
    print(f"model_outputs.start_logits shape is {model_outputs.start_logits.shape}")
    pred_probs = tuple(torch.nn.functional.softmax(model_output, dim=-1) for model_output in [model_outputs.start_logits, model_outputs.end_logits])
    pred_probs_argmax = tuple(prob.argmax(dim=-1) for prob in pred_probs)
    reverse_start_end_indices = pred_probs_argmax[0]>pred_probs_argmax[1]
    pred_pos = torch.stack(pred_probs_argmax, dim=-1)
    pred_pos[:,0][reverse_start_end_indices] = pred_probs_argmax[1][reverse_start_end_indices]
    pred_pos[:,1][reverse_start_end_indices] = pred_probs_argmax[0][reverse_start_end_indices]
    for row_doc_i in range(pred_pos.shape[0]):
        srctokens = batchdata['input_ids'][row_doc_i]
        start = pred_pos[row_doc_i,0]
        end = pred_pos[row_doc_i,1]
        incre_num_by = 0
        while True:
            span = torch.arange(start, end+1)
            span = slice_list_by_list_of_int(srctokens, span.tolist())
            span = get_data_instance.tokenizer.decode(span)
            matches = match_doc_id_pattern.findall(span)
            if matches or (start==0 and end>=max_seq_length) or incre_num_by>scan_ans_windowsize_limit:
                break
            else:
                incre_num_by += 5
                start = 0 if (start - incre_num_by)<=0 else (start-incre_num_by)
                end = 0 if (end + incre_num_by)<=0 else (end+incre_num_by)
        if not matches:
            pass
        else:
            pred_catg, pred_article_num = matches[0].split("_")
            func_batch_predictions_for_competition.append({
                "qid": batchdata['qid'][row_doc_i],
                "category": pred_catg,
                "retrieve": pred_article_num,
                "incre_num_by": incre_num_by
            })
            print(f"{batchdata['qid'][row_doc_i]} done!")
    # print(f"batch_predictions_for_competition {func_batch_predictions_for_competition}")
    return func_batch_predictions_for_competition

print("dataloader complete")
overallpredictions = []
with torch.no_grad():
    print(f"no_grad")
    for iter_i, orig in enumerate(iterds_validation_batched):
        print(f"iter_i {iter_i}")
        # print(f"orig is {orig}")
        inputs = {key: val for key, val in orig.items() if key in ['input_ids', 'attention_mask']}
        batch = data_collator(inputs)
        batch = {key:value.to(device) for key,value in batch.items()}
        outputs = model(**batch)
        towrite_dicts = model_output_arrange(outputs, orig, parserargs_dict['scan_ans_windowsize'])
        overallpredictions.extend(towrite_dicts)
        target_file_name_suffix = f"qid{orig['qid'][0]}_iter{iter_i}_checkpoint{parserargs_dict['checkpoint_num']}.json"
        target_file_name = writepath/target_file_name_suffix
        with target_file_name.open('w', encoding='utf-8') as json_file:
            json.dump(towrite_dicts, json_file, ensure_ascii=False, indent=4)
        if parserargs_dict['requrl'] is not None:
            try:
                reqdict = {'filename':target_file_name_suffix,'towrite_dicts':towrite_dicts}
                requests.post(parserargs_dict['requrl'], json=reqdict)
            except Exception as e:
                print(f"sending req failed for {e}")


target_file_name = writepath/parserargs_dict['ground_truths_filename']
overallpredictions = {"ground_truths":overallpredictions}
with target_file_name.open('w', encoding='utf-8') as json_file:
    json.dump(overallpredictions, json_file, ensure_ascii=False, indent=4)

366 done!
iter_i 256
model_outputs.start_logits shape is torch.Size([1, 36833])
367 done!
iter_i 257
model_outputs.start_logits shape is torch.Size([1, 7713])
368 done!
iter_i 258
model_outputs.start_logits shape is torch.Size([1, 16337])
369 done!
iter_i 259
model_outputs.start_logits shape is torch.Size([1, 16171])
370 done!
iter_i 260
model_outputs.start_logits shape is torch.Size([1, 38114])
371 done!
iter_i 261
model_outputs.start_logits shape is torch.Size([1, 33810])
372 done!
iter_i 262
model_outputs.start_logits shape is torch.Size([1, 13483])
373 done!
iter_i 263
model_outputs.start_logits shape is torch.Size([1, 7022])
374 done!
iter_i 264
model_outputs.start_logits shape is torch.Size([1, 40390])
375 done!
iter_i 265
model_outputs.start_logits shape is torch.Size([1, 8824])
376 done!
iter_i 266
model_outputs.start_logits shape is torch.Size([1, 12612])
377 done!
iter_i 267
model_outputs.start_logits shape is torch.Size([1, 35598])
378 done!
iter_i 268
model_outputs.start_lo

In [ ]:
!python /content/drive/MyDrive/models/inference_simpleqa.py --batch_size=2 --question_filename=questions_preliminary.json --write_path=/content/drive/MyDrive/models/predictions --df_start_id=500 --checkpoint_num=120 --ground_truths_filename=checkpoint120_all.json

2024-11-09 06:03:43.174839: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-09 06:03:43.211921: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-09 06:03:43.221789: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-09 06:03:43.245422: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-09 06:03:44.954411: W tensorflow/comp

2024-11-05 22:04:17.670026: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-05 22:04:17.689880: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-05 22:04:17.695906: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-05 22:04:17.710501: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-05 22:04:18.853075: W tensorflow/compiler/tf2

# Hyperparameter Searching

In [ ]:
import ast
import subprocess
import optuna

def objective(trial):
    # Define your hyperparameters to tune
    parameters = {
      'batch_size':trial.suggest_int('batch_size', 1, 2),
      'learning_rate':trial.suggest_float('learning_rate', 2e-5, 3e-4),
      'gradient_accumulation_steps':trial.suggest_int('gradient_accumulation_steps', 1, 10),
      'torch_empty_cache_steps':trial.suggest_int('torch_empty_cache_steps', 1, 10),
      'peft_r':trial.suggest_int('peft_r', 0, 1024),
      'fp16':trial.suggest_categorical('fp16', [True, False]),
      'load_in_4bit':trial.suggest_categorical('load_in_4bit', [False, True]),
      'optim':trial.suggest_categorical('optim', ['adamw_torch','adamw_bnb_8bit','adafactor']),
      'weight_decay':trial.suggest_float('weight_decay', 0.01, 0.1),
      'model_name':trial.suggest_categorical('model_name',
      ["meta-llama/Llama-3.2-1B-Instruct","Qwen/Qwen2.5-0.5B-Instruct",
        "Qwen/Qwen2.5-1.5B-Instruct","meta-llama/Llama-3.2-3B-Instruct"]
      ),
    }
    if parameters['peft_r']!=0:
      parameters['peft_lora_alpha'] = trial.suggest_int('peft_lora_alpha', 0, 1024)
    if parameters['fp16']:
      parameters['fp16_opt_level'] = trial.suggest_categorical('fp16_opt_level', ['O0','O1','O2','O3'])
    if not parameters['load_in_4bit']:
      parameters['load_in_8bit'] = trial.suggest_categorical('load_in_8bit', [False, True])
    if parameters['load_in_4bit'] or parameters['load_in_8bit']:
      parameters["llm_int8_has_fp16_weight"] = trial.suggest_categorical('llm_int8_has_fp16_weight', [False, True])
      parameters["bnb_4bit_compute_dtype"] = trial.suggest_categorical('bnb_4bit_compute_dtype', ['fp32','bf16'])
      parameters["bnb_4bit_quant_type"] = trial.suggest_categorical('bnb_4bit_quant_type', ['fp4','nf4'])
      parameters['bnb_4bit_use_double_quant'] = trial.suggest_categorical('bnb_4bit_use_double_quant', [False, True])

    command = ["python", "/content/drive/MyDrive/models/training_simpleqa.py",]
    for key, value in parameters.items():
      command.append(
          f"--{key}={str(value)}"
      )
    joined_command = " ".join(command)
    print(f"command is: {joined_command}")
    metric_inf = {}
    try:
        process = subprocess.Popen(
            command, stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT, text=True, bufsize=1
        )
        captured_output = []
        metric_inf_str = ""
        with process.stdout:
            for line in iter(process.stdout.readline, ''):
                captured_output.append(line)
                print(f"   {line}")
                if line.find('eval_loss')!=-1 and line.find('eval_precision')!=-1 and line.find('eval_recall')!=-1:
                    metric_inf_str = line
        # process.communicate()
        metric_inf_str = metric_inf_str.strip()
        if metric_inf_str!='':
            metric_inf = ast.literal_eval(metric_inf_str)
            # eval_loss eval_precision eval_recall
    except Exception as e:
        print(f"An error occurred: {e}")
        # metric_inf['eval_loss'] = 9999999999999999999
        raise optuna.TrialPruned(f"Trial pruned due to error: {e}")
    if 'eval_loss' not in metric_inf:
      raise optuna.TrialPruned(f"Trial pruned due to error: no eval_loss")

    return metric_inf['eval_loss'], metric_inf['eval_samples_per_second'] #, metric_inf['eval_precision']

In [ ]:
# Create a study and optimize it
storage = optuna.storages.RDBStorage(url="sqlite:////content/drive/MyDrive/models/optuna_trials.db", heartbeat_interval=60, grace_period=120)
study = optuna.create_study(
    study_name='financialRAG_doublemin',
    storage=storage,
    load_if_exists=True,
    directions=['minimize','minimize']
)
study.set_user_attr('contributors', ['TJ'])
optuna.logging.set_verbosity(optuna.logging.DEBUG)
study.optimize(objective, n_trials=300)

print("Best hyperparameters:", study.best_params)

[I 2024-11-07 05:43:11,057] Using an existing study with name 'financialRAG_doublemin' instead of creating a new one.
/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py:186: ExperimentalWarning: fail_stale_trials is experimental (supported from v2.9.0). The interface can change in the future.
  optuna.storages.fail_stale_trials(study)
[D 2024-11-07 05:43:11,135] Trial 41 popped from the trial queue.


command is: python /content/drive/MyDrive/models/training_simpleqa.py --batch_size=1 --learning_rate=0.00020715556360024592 --gradient_accumulation_steps=8 --torch_empty_cache_steps=6 --peft_r=997 --fp16=True --load_in_4bit=False --optim=adamw_bnb_8bit --weight_decay=0.08244680286182575 --model_name=Qwen/Qwen2.5-1.5B-Instruct --peft_lora_alpha=796 --fp16_opt_level=O3 --load_in_8bit=False
   2024-11-07 05:43:21.475250: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered

   2024-11-07 05:43:21.708309: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered

   2024-11-07 05:43:21.774771: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS 

[I 2024-11-07 06:39:12,351] Trial 41 finished with values: [2.822599172592163, 0.442] and parameters: {'batch_size': 1, 'learning_rate': 0.00020715556360024592, 'gradient_accumulation_steps': 8, 'torch_empty_cache_steps': 6, 'peft_r': 997, 'fp16': True, 'load_in_4bit': False, 'optim': 'adamw_bnb_8bit', 'weight_decay': 0.08244680286182575, 'model_name': 'Qwen/Qwen2.5-1.5B-Instruct', 'peft_lora_alpha': 796, 'fp16_opt_level': 'O3', 'load_in_8bit': False}.
/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py:186: ExperimentalWarning: fail_stale_trials is experimental (supported from v2.9.0). The interface can change in the future.
  optuna.storages.fail_stale_trials(study)


command is: python /content/drive/MyDrive/models/training_simpleqa.py --batch_size=2 --learning_rate=0.00014778501823750886 --gradient_accumulation_steps=8 --torch_empty_cache_steps=2 --peft_r=135 --fp16=False --load_in_4bit=False --optim=adamw_torch --weight_decay=0.020316957796791307 --model_name=Qwen/Qwen2.5-0.5B-Instruct --peft_lora_alpha=416 --load_in_8bit=True --llm_int8_has_fp16_weight=True --bnb_4bit_compute_dtype=bf16 --bnb_4bit_quant_type=nf4 --bnb_4bit_use_double_quant=False
   2024-11-07 06:39:19.318961: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered

   2024-11-07 06:39:19.346324: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered

   2024-11-07 06:39:19.353174: E external/local_xla/xla/stream_executor/cuda/cud

[I 2024-11-07 06:39:53,867] Trial 42 pruned. Trial pruned due to error: no eval_loss
/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py:186: ExperimentalWarning: fail_stale_trials is experimental (supported from v2.9.0). The interface can change in the future.
  optuna.storages.fail_stale_trials(study)


...

command is: python /content/drive/MyDrive/models/training_simpleqa.py --batch_size=2 --learning_rate=0.00014233459806555114 --gradient_accumulation_steps=2 --torch_empty_cache_steps=4 --peft_r=402 --fp16=True --load_in_4bit=False --optim=adafactor --weight_decay=0.037384954043984876 --model_name=Qwen/Qwen2.5-0.5B-Instruct --peft_lora_alpha=828 --fp16_opt_level=O1 --load_in_8bit=True --llm_int8_has_fp16_weight=True --bnb_4bit_compute_dtype=fp32 --bnb_4bit_quant_type=nf4 --bnb_4bit_use_double_quant=False
   2024-11-07 09:00:03.353272: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered

   2024-11-07 09:00:03.374577: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered

   2024-11-07 09:00:03.381422: E external/local_xla/xla/stream_

[I 2024-11-07 09:00:12,736] Trial 58 pruned. Trial pruned due to error: no eval_loss
/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py:186: ExperimentalWarning: fail_stale_trials is experimental (supported from v2.9.0). The interface can change in the future.
  optuna.storages.fail_stale_trials(study)


command is: python /content/drive/MyDrive/models/training_simpleqa.py --batch_size=1 --learning_rate=0.0002914201318358158 --gradient_accumulation_steps=1 --torch_empty_cache_steps=8 --peft_r=880 --fp16=False --load_in_4bit=False --optim=adafactor --weight_decay=0.024058085411881668 --model_name=meta-llama/Llama-3.2-1B-Instruct --peft_lora_alpha=402 --load_in_8bit=True --llm_int8_has_fp16_weight=False --bnb_4bit_compute_dtype=fp32 --bnb_4bit_quant_type=fp4 --bnb_4bit_use_double_quant=True
   2024-11-07 09:00:16.732380: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered

   2024-11-07 09:00:16.752777: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered

   2024-11-07 09:00:16.759118: E external/local_xla/xla/stream_executor/cuda/

[I 2024-11-07 09:00:37,858] Trial 59 pruned. Trial pruned due to error: no eval_loss
/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py:186: ExperimentalWarning: fail_stale_trials is experimental (supported from v2.9.0). The interface can change in the future.
  optuna.storages.fail_stale_trials(study)


command is: python /content/drive/MyDrive/models/training_simpleqa.py --batch_size=1 --learning_rate=0.00010131281787056267 --gradient_accumulation_steps=4 --torch_empty_cache_steps=9 --peft_r=283 --fp16=True --load_in_4bit=True --optim=adamw_torch --weight_decay=0.09585633640576324 --model_name=Qwen/Qwen2.5-0.5B-Instruct --peft_lora_alpha=408 --fp16_opt_level=O2 --llm_int8_has_fp16_weight=False --bnb_4bit_compute_dtype=fp32 --bnb_4bit_quant_type=fp4 --bnb_4bit_use_double_quant=False
   2024-11-07 09:00:41.889752: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered

   2024-11-07 09:00:41.914759: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered

   2024-11-07 09:00:41.921228: E external/local_xla/xla/stream_executor/cuda/cuda_